# About: Notebook環境Prerequisite

Notebook環境が満たすべき状態を明示的に定義する。
自分のNotebook環境にこのNotebookをコピーし、読み進めていって、条件を満たしているかどうかを確認する。

条件には推奨の度合いによりレベルがあり、 **A=必須、B=推奨、C=任意** というレベルを設定してある。

このNotebookを確認することで、全ホストが **Seedingが済んだ状態である** かどうかを確認することができる。

# Ansibleの設定

Notebook環境は、以下の条件を満たしている必要がある。

## Ansibleがインストールされていること - A

以下のコマンド実行がエラーとならず、インストールされているAnsibleのバージョンが表示されること。

In [1]:
!ansible --version

ansible 2.1.0.0
  config file = /etc/ansible/ansible.cfg
  configured module search path = Default w/o overrides


ansibleコマンドがインストールされていない場合は、[Installing the Control Machine](http://docs.ansible.com/ansible/intro_installation.html#installing-the-control-machine)を参考に、Ansibleをインストールする。

## AnsibleでDefault Moduleが利用可能であること - A

DefaultのModuleが使えればよい。特にカスタマイズすべき項目はなし。*...将来は腐っているモジュールは差し替えたい。*

# Ansible - Binding対象間の設定

このNotebook環境にインストールされているAnsibleは、Binding対象となるホストに対して通信、操作可能な状態になっている必要がある。

## Default Inventoryが設定されていること - B

以下のコマンド実行がエラーとならず、 `inventory = (Default Inventoryのパス)` が設定されていること。

> これを指定しないと、運用Notebookのansible実行ごとにInventoryを指定する必要がある・・・あまりNotebookには **Notebookが動作する環境に依存した項目を書きたくない** ので必須としている。

In [2]:
!cat ${ANSIBLE_CONFIG:-/etc/ansible/ansible.cfg} | grep --color=always -A 2 -B 2 "^inventory"

private_key_file = /root/.ssh/ansible_id_rsa
ansible_ssh_user = ansible
inventory = /notebooks/inventory


[Configuration file](http://docs.ansible.com/ansible/intro_configuration.html) を参考にしながら、Default Inventoryを指定しておく。Default Inventoryを指定しない場合は、対象とのBindingの際にInventoryの指定を加えること。

## Inventoryに指定されたホストに到達可能なこと - A

AnsibleのInventoryに指定された全ホストに対して、この環境からアクセス可能であること。

- 以下のコマンド実行がエラーもしくは警告(`provided hosts list is empty`)とならないこと
- Inventoryに登録されているホストに関して、すべてSUCCESSとなること

In [3]:
!ansible -m ping all

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.105 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.59 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


このNotebook環境のAnsibleから、ホストへの認証方法については特に問わない。

> 我々(NIIクラウド運用チーム)の環境ではkeypairによる認証を使ってAnsibleから各ホストにアクセスをしている。
> そのため、以下のpublic keyをBinding対象ホストの `~/.ssh/authorized_keys` に設定している。

In [4]:
!cat ~/.ssh/ansible_id_rsa.pub

ssh-rsa XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ansible@XXXXXXXXXXXX


## 適切なユーザで認証されること - B

Ansibleからホストへの認証は、一般ユーザとして認証されるようにしておく。

> 我々(NIIクラウド運用チーム)の環境では、ユーザ ansible を作って対応している。

In [5]:
!ansible -a 'whoami' all

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
ansible

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
ansible

XXX.XXX.XXX.59 | SUCCESS | rc=0 >>
ansible



## Binding対象ホストにてsudo可能であること - A

Inventoryに記述された全ホストに関してbecome, become-userが可能であることを確認しておく。

- 以下のコマンド実行がエラーとならないこと
- Inventoryに登録されているホストに関して、すべてSUCCESSとなり、結果がrootとなること

In [6]:
!ansible --become --become-user root -a 'whoami' all

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
root

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
root

XXX.XXX.XXX.59 | SUCCESS | rc=0 >>
root



以上の項目が満たされていればOK。

# Binding対象の設定

Binding対象に関して、Notebook + Ansibleによる管理を適用する上で適切な設定になっていることを確認する。

## Ansibleによる操作ログが記録されていること - B

NotebookからBinding対象の操作をおこなった際に、過去おこなった操作を適切に参照できる必要がある。

> 我々の環境では、ansibleによる操作は `/var/log/secure` (CentOSの場合) `/var/log/auth.log` (Ubuntuの場合) に記録されている

In [7]:
!ansible -b -m shell -a 'grep ansible- /var/log/* | tail' all

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
/var/log/auth.log:Jun 17 23:02:22 ubuntu sudo:  ansible : TTY=pts/3 ; PWD=/home/ansible ; USER=root ; COMMAND=/bin/sh -c echo BECOME-SUCCESS-bvlvlprywjftoofuddcvycadctvlxlpt; LANG=en_US.UTF-8 LC_ALL=en_US.UTF-8 LC_MESSAGES=en_US.UTF-8 /usr/bin/python /home/ansible/.ansible/tmp/ansible-tmp-1466172142.43-67445864722369/command; rm -rf "/home/ansible/.ansible/tmp/ansible-tmp-1466172142.43-67445864722369/" > /dev/null 2>&1
/var/log/auth.log:Jun 17 23:02:27 ubuntu sudo:  ansible : TTY=pts/3 ; PWD=/home/ansible ; USER=root ; COMMAND=/bin/sh -c echo BECOME-SUCCESS-hsyrmwfqwdfrolqqfinhmdyyuhavfoxt; LANG=en_US.UTF-8 LC_ALL=en_US.UTF-8 LC_MESSAGES=en_US.UTF-8 /usr/bin/python /home/ansible/.ansible/tmp/ansible-tmp-1466172147.34-2143486004272/command; rm -rf "/home/ansible/.ansible/tmp/ansible-tmp-1466172147.34-2143486004272/" > /dev/null 2>&1
/var/log/auth.log:Jun 17 23:02:48 ubuntu sudo:  ansible : TTY=pts/3 ; PWD=/home/ansible ; USER=root ; COMMAND=/bin/sh -c 

### ログファイルが適切にrotateされていること - B

操作の記録がディスクを圧迫しないことを確認しておく。ログは短期的な(Ansibleによる)操作の記録として参照可能にしておくべきものという前提。長期的には、Notebookを証跡としてアーカイブしておくべき。

## 共通の方法でホストの状況が確認できること - C

Notebookによる操作実施時は、適当なチェックポイントごとに状況を記録することで、後戻りしたくなった時に戻れる(戻ったことが確認できる)ようにする。

### OSのバージョン確認

lsb_releaseを使う。

In [8]:
!ansible -a 'lsb_release -a' all

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Distributor ID:	Ubuntu
Description:	Ubuntu 14.04.4 LTS
Release:	14.04
Codename:	trustyNo LSB modules are available.

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
LSB Version:	:base-4.0-amd64:base-4.0-noarch:core-4.0-amd64:core-4.0-noarch:graphics-4.0-amd64:graphics-4.0-noarch:printing-4.0-amd64:printing-4.0-noarch
Distributor ID:	CentOS
Description:	CentOS release 6.8 (Final)
Release:	6.8
Codename:	Final

XXX.XXX.XXX.59 | SUCCESS | rc=0 >>
Distributor ID:	Ubuntu
Description:	Ubuntu 14.04.4 LTS
Release:	14.04
Codename:	trustyNo LSB modules are available.



### ファイル構成の確認

treeコマンドを使う。

In [9]:
!ansible -a 'tree /tmp' all

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
/tmp
└── ansible_bxzXfW
    ├── ansible_modlib.zip
    └── ansible_module_command.py

1 directory, 2 files

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
/tmp
├── ansible_DgRpfV
│   ├── ansible_modlib.zip
│   └── ansible_module_command.py
├── CentOS-6.8-x86_64-minimal.iso
├── centos6.ks.cfg
└── ubuntu14.04.ks.cfg

1 directory, 5 files

XXX.XXX.XXX.59 | SUCCESS | rc=0 >>
/tmp
└── ansible_BYOvNB
    ├── ansible_modlib.zip
    └── ansible_module_command.py

1 directory, 2 files



## リソースの使用状況変化をモニタリングしていること - C

Binding対象を操作した際に、予期せぬディスク使用量増加やサービス異常終了に気づけるよう、状況変化をモニタリングしていることが望ましい。